<a href="https://colab.research.google.com/github/Mikhail-068/NetOptic/blob/master/Tretyakov_Aleksandr/%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Подключение модуля для загрузки данных из облака
import gdown
# Модуль для работы с файлами операционной системы
import os
from bs4 import BeautifulSoup
# Модуль для вывода данных в читабельном виде
from pprint import pprint
import pandas as pd
import numpy as np

# Скачивание базы

### Скачивание с сервера заказчика

Чтобы постоянно не долбиться на сервер, скачаем базу один раз, сохраним её на диск и потом будем обращаться к файлу.

In [ ]:
# Загрузка zip-архива с датасетом из облака на диск виртуальной машины colab
base_file_name = 'yandexmarket.yml'
gdown.download('https://www.netoptik.ru/yandexmarket.yml', output=base_file_name)
base_path = os.path.join('/content', base_file_name)

Downloading...
From: https://www.netoptik.ru/yandexmarket.yml
To: /content/yandexmarket.yml
13.5kB [00:00, 30.8MB/s]                   


### Скачивание с Google.Drive

In [ ]:
base_file_name = 'yandexmarket.yml'
gdown.download('https://drive.google.com/uc?id=12Feyswg1yBGfUvJKLvWtV-OIP-HbUqME', output=base_file_name)
base_path = os.path.join('/content', base_file_name)

Downloading...
From: https://drive.google.com/uc?id=12Feyswg1yBGfUvJKLvWtV-OIP-HbUqME
To: /content/yandexmarket.yml
100%|██████████| 4.55M/4.55M [00:00<00:00, 15.9MB/s]


# Парсинг

In [ ]:
!pip install beautifulsoup4==4.12.2 lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2


In [ ]:
# читаем файл
with open(base_path, 'r') as file:
    xml_file = file.read()

In [ ]:
# Получаем объект bs
soup = BeautifulSoup(xml_file, 'lxml')
offers = soup.findAll("offer")

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


## Эксперименты с парсером

In [ ]:
# Номер ордера, который мы ходим посмотреть
OFFER_NUM = 12
one_offer = offers[OFFER_NUM]
print(one_offer)

<offer id="11795">
<categoryid>177</categoryid>
<name>Оправа Gucci, GG 1006, 4GX</name>
<url>https://www.netoptik.ru/frames/men-frames/oprava-gucci-gg1006-4gx</url>
<price>7500</price>
<currencyid>RUB</currencyid>
<vendor>GUCCI</vendor>
<model>Оправа Gucci, GG 1006, 4GX</model>
<picture>https://www.netoptik.ru/image/cache/catalog/Gucci%20(ЮД)/gucci-gg1006-4gx-s2-1%20(2)-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/catalog/Gucci%20(ЮД)/gucci-gg1006-4gx-s2-2%20(1)-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/catalog/Gucci%20(ЮД)/gucci-gg1006-4gx-s2-3%20(3)-600x600.jpg</picture>
<param name="Категория"/>Мужские оправы для очков
<param name="Ширина оправы (мм)"/>135
<param name="Ширина линзы (мм)"/>54
<param name="Высота линзы (мм)"/>36
<param name="Мост (мм)"/>16
<param name="Длина заушника (мм)"/>140
<param name="Материал "/>Комбинированный
<param name="Стиль"/>Мужские
<param name="Строение оправы"/>Ободковая
<param name="Цвет оправы"/>Черепахо

In [ ]:
# Посмотрим названия всех доступных параметров
for param in one_offer.select('param'):
    print(param.attrs['name'])

Категория
Ширина оправы (мм)
Ширина линзы (мм)
Высота линзы (мм)
Мост (мм)
Длина заушника (мм)
Материал 
Стиль
Строение оправы
Цвет оправы
Форма очков


## Парсим базу

In [ ]:
# Зададим список данных (тэгов), которые нам нужны
TAGS_LIST = ['name', 'picture']
# Зададим список параметров, которые нам нужны
PARAMS_LIST = ['Категория', 'Материал']

In [ ]:
data_list = []

for offer in offers:
    new_line = {}

    # Добавляем значения нужных нам тэгов
    for tag in TAGS_LIST:
        new_line[tag] = offer.find(tag).getText()

        if tag == 'picture':
            result = []
            for pic in offer.find_all(tag):
                result.append(pic.getText())
            new_line[tag] = '\n'.join(result)


    # Добавляем значения нужных нам параметров
    for needed_param in PARAMS_LIST:
        for param in offer.find_all('param'):
            if needed_param in param.attrs['name'].strip():
                new_line[needed_param] = param.next_element.replace('\n', '').strip()

    # добавляем данные в список data_list
    data_list.append(new_line)

In [ ]:
# Добавляем данные в датафрейм
base_df = pd.DataFrame(data_list)

In [ ]:
display(base_df)

,name,picture,Категория,Материал
0,"Оправа Balenciaga, BAL 0108, 8O0",https://www.netoptik.ru/image/cache/data/med_o...,Оправы для очков,комбинированный
1,"Оправа Boss Orange, BO 0329, YZ4",https://www.netoptik.ru/image/cache/catalog/Bo...,Мужские оправы для очков,Металл
2,"Оправа Boss Hugo Boss, BOSS 1312, 003",https://www.netoptik.ru/image/cache/catalog/Bo...,Оправы для очков,Комбинированный
3,"Оправа Boss Hugo Boss, BOSS 1246, CNI",https://www.netoptik.ru/image/cache/catalog/Bo...,Оправы для очков,Металл
4,"Оправа Carrera, CA 6660, VBP",https://www.netoptik.ru/image/cache/data/carre...,Женские оправы для очков,металл
...,...,...,...,...
2448,"Оправа FILOS, FF1300, 04",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл
2449,"Оправа FILOS, FF1300, 05",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл
2450,"Оправа FILOS, FF1303, 03",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл
2451,"Оправа FILOS, FF1303, 01",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл


In [ ]:
display(base_df['Материал'].unique())

array(['комбинированный', 'Металл', 'Комбинированный', 'металл',
       'Пластик', 'пластик', nan, 'Трайвекс', 'Силикон',
       'Пластик, ударопрочный', 'титан', 'Поликарбонат', 'Титан',
       'Стекло', 'Silflex', 'силикон', 'Комбинированные'], dtype=object)

In [ ]:
# Получаем уникальные значения из столбца 'Категория' в DataFrame
unique_values = base_df['Категория'].fillna('нет данных').unique()

# Сортируем значения по алфавиту, игнорируя регистр букв
sorted_values = sorted(unique_values, key=str.lower)

# Выводим отсортированные значения
pprint(sorted_values)

['SALE',
 'Аксессуары',
 'Бифокальные линзы',
 'Водительские очки',
 'Готовые очки',
 'Детские',
 'Детские очки',
 'Женские оправы для очков',
 'женские очки',
 'Женские очки',
 'Женскиеоправы для очков',
 'Женский',
 'Компьютерные линзы',
 'Компьютерные очки',
 'Лентикуляры для высоких диоптрий',
 'Линзы для детей',
 'Линзы для очков',
 'Линзы с поддержкой аккомадации',
 'Мужские оправы для очков',
 'Мужские очки',
 'Мужскиеоправы для очков',
 'нет данных',
 'Однофокальные линзы',
 'Оправы',
 'Оправы для очков',
 'Оправы оправы для очков',
 'Офисные линзы',
 'Очки для плавания',
 'Очки на очки',
 'Очки спецназначения',
 'Очки тренажеры',
 'Пенсне',
 'Половинки для чтения',
 'Прогрессивные линзы',
 'Солнцезащитные очки',
 'Спецназначения',
 'Спортивные',
 'Спортивные очки',
 'Унисекс',
 'Фотохромные линзы']


In [ ]:
# Получаем уникальные значения из столбца 'Материал' в DataFrame
unique_values = base_df['Материал'].fillna('нет данных').unique()

# Сортируем значения по алфавиту, игнорируя регистр букв
sorted_values = sorted(unique_values, key=str.lower)

# Выводим отсортированные значения
pprint(sorted_values)

['Silflex',
 'Комбинированные',
 'комбинированный',
 'Комбинированный',
 'Металл',
 'металл',
 'нет данных',
 'Пластик',
 'пластик',
 'Пластик, ударопрочный',
 'Поликарбонат',
 'Силикон',
 'силикон',
 'Стекло',
 'титан',
 'Титан',
 'Трайвекс']


In [ ]:
base_df['Материал'].str.lower().value_counts().sort_index()

silflex                     8
комбинированные             5
комбинированный           624
металл                    429
пластик                  1175
пластик, ударопрочный       1
поликарбонат               23
силикон                    95
стекло                      1
титан                      68
трайвекс                    5
Name: Материал, dtype: int64

In [ ]:
base_df['Материал'].value_counts().sort_index()

Silflex                    8
Комбинированные            5
Комбинированный          417
Металл                   317
Пластик                  965
Пластик, ударопрочный      1
Поликарбонат              23
Силикон                   71
Стекло                     1
Титан                     65
Трайвекс                   5
комбинированный          207
металл                   112
пластик                  210
силикон                   24
титан                      3
Name: Материал, dtype: int64

## Причешем данные

In [ ]:
# nan заменим на 'нет данных'
base_df['Материал'] = base_df['Материал'].fillna('нет данных')

In [ ]:
# Удалим ненужные данные
delete_list = ['Silflex', 'Пластик, ударопрочный', 'Поликарбонат',
               'Силикон', 'силикон', 'Стекло', 'титан', 'Титан',
               'Трайвекс']

In [ ]:
# удаляем строки, у которых значение в столбце "Материал" равно значению из списка delete_list
base_df = base_df[~base_df['Материал'].isin(delete_list)]

In [ ]:
# Смотрим, что получилось
print(base_df['Материал'].unique())

['комбинированный' 'Металл' 'Комбинированный' 'металл' 'Пластик' 'пластик'
 'нет данных' 'Комбинированные']


In [ ]:
# Словарь замены значений
replace_dict = {
    'комбинированный': ['Комбинированные', 'Комбинированный'],
    'металл': ['Металл'],
    'пластик': ['Пластик']
    #'silflex': ['Silflex'],
    #'поликарбонат': ['Поликарбонат'],
    #'силикон': ['Силикон'],
    #'стекло': ['Стекло'],
    #'титан': ['Титан'],
    #'трайвекс': ['Трайвекс']
}

In [ ]:
# заменяем значения в столбце "Материал" по словарю replace_dict
for key, values in replace_dict.items():
    base_df['Материал'] = base_df['Материал'].replace(values, key)

<ipython-input-24-7c75b2de8f38>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df['Материал'] = base_df['Материал'].replace(values, key)


In [ ]:
# Смотрим, что получилось
print(base_df['Материал'].unique())

['комбинированный' 'металл' 'пластик' 'нет данных']


In [ ]:
base_df['Материал'].value_counts().sort_index()

комбинированный     629
металл              429
нет данных           19
пластик            1175
Name: Материал, dtype: int64

In [ ]:
# Сохраним данные в файл формата csv
base_df.to_csv('materials_data.csv', index=False)

## Соберём по каждому классу отдельную папку с изображениями

In [ ]:
# import os
# import pandas as pd
import requests
import time

# Создание папки materials_pictures, если её ещё нет
if not os.path.exists('materials_pictures'):
    os.makedirs('materials_pictures')

# Создание папок для каждого материала внутри materials_pictures,
# если их ещё нет
materials = list(base_df['Материал'].unique())
for material in materials:
    if not os.path.exists(os.path.join('materials_pictures', material)):
        os.makedirs(os.path.join('materials_pictures', material))

# Чтение данных из DataFrame
base_df = pd.read_csv('materials_data.csv')

# Проход по строкам DataFrame и скачивание картинок
start_time = time.time() # засекаем начало времени
for index, row in base_df.iterrows():
    material = row['Материал']
    picture_links = row['picture'].split('\n')

    for link in picture_links:
        try:
            response = requests.get(link)
        except:
            pass
        else:
            if response.status_code == 200:
                filename = os.path.join('materials_pictures', material,
                                        os.path.basename(link))
                with open(filename, 'wb') as f:
                    f.write(response.content)
            else:
                print(f'Ошибка загрузки изображения: {link}')

# Вывод времени выполнения
end_time = time.time() # засекаем конец времени
print(f'Время выполнения: {end_time - start_time:.2f} секунд')

MissingSchema: ignored

До ошибки проработала 50 минут.

In [ ]:
import shutil

# Создание архива папки materials_pictures
result = shutil.make_archive('materials_pictures_archive', 'zip',
                             'materials_pictures')
print(f'Архив картинок скачать здесь:\n{result}')

Архив картинок скачать здесь:
/content/materials_pictures_archive.zip
